In [2]:
"""
generate_pollen.py

Generates fictional but plausible daily pollen measurements for each zone
found in `indice_pollen_bretagne_epci.csv` (if present) or synthesizes 62 zones,
for the date range 2020-01-01 .. 2025-12-31.

Output: indice_pollen_bretagne_epci_generated.csv
"""

import os
import math
import random
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from tqdm import tqdm

# ----------------------
# Config
# ----------------------
INPUT_CSV = "indice_pollen_bretagne_epci.csv"
OUTPUT_CSV = "indice_pollen_bretagne_epci_generated.csv"
START_DATE = datetime(2020, 1, 1)
END_DATE = datetime(2025, 12, 31)
RNG_SEED = 42  # change or set to None for non-deterministic results
N_SYNTHETIC_ZONES = 62
TYPE_ZONE_DEFAULT = "EPCI"
SOURCE_DEFAULT = "Air Breizh"
EPSG_DEFAULT = "2154"  # per your sample
COUL_MAP = {1: "#50F0E6", 2: "#50CCAA", 3: "#F0C050", 4: "#FF5050", 0: "#FFFFFF"}
LIB_QUAL_MAP = {1: "Bon", 2: "Moyen", 3: "Dégradé", 4: "Mauvais", 0: "No Data"}

# species order in CSV:
SPECIES = ["ambroisie", "aulne", "armoise", "bouleau", "graminees", "olivier"]

# ----------------------
# Utilities
# ----------------------
if RNG_SEED is not None:
    random.seed(RNG_SEED)
    np.random.seed(RNG_SEED)

def date_range(start, end):
    d = start
    while d <= end:
        yield d
        d += timedelta(days=1)

def month_frac(dt):
    """Return a fractional month index 1..12 for season functions."""
    return dt.month + (dt.day - 1) / 31.0

# seasonal "activity" functions for species (0..1)
def season_activity(species, dt):
    m = dt.month
    # Rough seasonal peaks (based on typical pollen seasons in temperate Europe)
    if species == "bouleau":  # birch: spring (Mar-Apr-May)
        peak = 4
        width = 2.0
    elif species == "graminees":  # grasses: late spring to summer (May-Jun-Jul)
        peak = 6
        width = 3.0
    elif species == "aulne":  # alder: late winter - early spring (Feb-Mar-Apr)
        peak = 3
        width = 1.8
    elif species == "armoise":  # mugwort / artemisia: late summer - early autumn (Aug-Sep)
        peak = 8.5
        width = 1.5
    elif species == "ambroisie":  # ragweed: late summer - autumn (Aug-Sep)
        peak = 9
        width = 1.2
    elif species == "olivier":  # olive: localized, low in Brittany; small year-round baseline with tiny spring peak
        peak = 5
        width = 1.0
    else:
        peak = 6
        width = 3.0

    # Use a wrapped Gaussian around months 1..12
    # Convert month to angle on circle
    angle = (m - peak) * (2 * math.pi / 12)
    val = math.exp(- (angle ** 2) / (2 * ( (width * 2*math.pi/12) ** 2)))
    # add small baseline noise
    baseline = 0.03 if species == "olivier" else 0.02
    return baseline + (1.0 - baseline) * val

def activity_to_code(activity_value, overall_quality_code):
    """
    Map an activity value (0..1) and overall quality to an integer code 1..4.
    We bias species code by activity and by the overall qualitative situation.
    """
    # combined score [0..1]
    # overweight the activity but include some noise and quality effect
    score = 0.7 * activity_value + 0.2 * (1 - (overall_quality_code - 1) / 3.0)  # better overall quality -> lower pollen
    score += random.uniform(-0.08, 0.08)
    # thresholds (tuned to produce many 1-2, some 3, few 4)
    if score < 0.25:
        return 1
    elif score < 0.55:
        return 2
    elif score < 0.80:
        return 3
    else:
        return 4

# ----------------------
# Load / discover zones
# ----------------------
if os.path.exists(INPUT_CSV):
    print(f"Reading zones from existing CSV {INPUT_CSV} ...")
    src = pd.read_csv(INPUT_CSV, dtype=str)
    # ensure columns we need exist; try to coerce coords to numeric if present
    for c in ["code_zone", "lib_zone", "x_wgs84", "y_wgs84", "x_reg", "y_reg", "epsg_reg"]:
        if c not in src.columns:
            print(f"Warning: input CSV missing column {c}; synthesizing zones instead.")
            src = None
            break
else:
    src = None

zones = []
if src is not None:
    # get unique zones by code_zone; keep first seen coordinates
    unique = src.drop_duplicates(subset=["code_zone"])
    # if there are more than 61 keep all
    for _, row in unique.iterrows():
        cz = str(row.get("code_zone", "")).strip()
        if cz == "" or pd.isna(cz):
            continue
        lib = row.get("lib_zone", cz)
        try:
            x_w = float(row.get("x_wgs84", np.nan))
            y_w = float(row.get("y_wgs84", np.nan))
        except Exception:
            x_w, y_w = np.nan, np.nan
        try:
            x_reg = float(row.get("x_reg", np.nan))
            y_reg = float(row.get("y_reg", np.nan))
        except Exception:
            x_reg, y_reg = x_w, y_w
        epsg = row.get("epsg_reg", EPSG_DEFAULT)
        zones.append({
            "code_zone": cz,
            "lib_zone": lib,
            "x_wgs84": x_w,
            "y_wgs84": y_w,
            "x_reg": x_reg,
            "y_reg": y_reg,
            "epsg_reg": epsg
        })

if len(zones) < N_SYNTHETIC_ZONES:
    print("Synthesizing zones (not enough existing zones found)...")
    zones = []
    # bounding box for Brittany approx lon [-5.5, -1.0], lat [47.0, 49.5]
    for i in range(N_SYNTHETIC_ZONES):
        lon = random.uniform(-5.5, -1.0)
        lat = random.uniform(47.0, 49.5)
        code = f"Z{1000 + i}"
        lib = f"Zone synthétique {i+1}"
        # for x_reg/y_reg use placeholder projected coords (not real projection)
        x_reg = 100000 + (lon + 6.0) * 50000 + random.uniform(-1000, 1000)
        y_reg = 6000000 + (lat - 46.0) * 10000 + random.uniform(-1000, 1000)
        zones.append({
            "code_zone": code,
            "lib_zone": lib,
            "x_wgs84": lon,
            "y_wgs84": lat,
            "x_reg": x_reg,
            "y_reg": y_reg,
            "epsg_reg": EPSG_DEFAULT
        })

print(f"Using {len(zones)} zones for generation.")

# ----------------------
# For each zone, define absence periods (no sampling)
# ----------------------
zone_absences = {}
for z in zones:
    # For each zone pick 1..4 absence intervals (to simulate station downtime / seasonal no-sampling)
    n_abs = random.choice([1,1,1,2])  # most zones have 1 absence block, some 2
    abs_periods = []
    total_days = (END_DATE - START_DATE).days + 1
    for _ in range(n_abs):
        length = random.randint(15, min(150, total_days//2))  # days
        # pick a random start
        start_offset = random.randint(0, max(0, total_days - length))
        astart = START_DATE + timedelta(days=start_offset)
        aend = astart + timedelta(days=length-1)
        abs_periods.append((astart, aend))
    # Merge overlapping intervals
    abs_periods_sorted = sorted(abs_periods, key=lambda x: x[0])
    merged = []
    for s,e in abs_periods_sorted:
        if not merged or s > merged[-1][1] + timedelta(days=1):
            merged.append([s,e])
        else:
            merged[-1][1] = max(merged[-1][1], e)
    zone_absences[z["code_zone"]] = merged

# ----------------------
# Generate per-day data
# ----------------------
out_rows = []
# We'll generate per zone to keep memory manageable
total_days = (END_DATE - START_DATE).days + 1
dates = [START_DATE + timedelta(days=i) for i in range(total_days)]

print("Generating daily measurements (this may take a little while)...")
for z in tqdm(zones, unit="zone"):
    cz = z["code_zone"]
    libz = z["lib_zone"]
    xw = z["x_wgs84"]
    yw = z["y_wgs84"]
    xr = z["x_reg"]
    yr = z["y_reg"]
    epsg = z.get("epsg_reg", EPSG_DEFAULT)

    # Define a zone-specific baseline pollution tendency (1..4) biased to 1-2
    baseline = random.choices([1,2,3,4], weights=[40,35,18,7])[0]

    # Slight long-term drift per year for variety
    drift = np.linspace(0, random.uniform(-0.2, 0.3), total_days)

    # Get absence intervals for quick check
    abs_intervals = zone_absences.get(cz, [])

    for idx, dt in enumerate(dates):
        # check absence
        is_absent = False
        for s,e in abs_intervals:
            if s <= dt <= e:
                is_absent = True
                break

        date_ech_str = dt.strftime("%Y/%m/%d 00:00:00")
        if is_absent:
            # all codes zero, No Data
            row = {
                "date_ech": date_ech_str,
                "code_qual": "0",
                "lib_qual": "No Data",
                "coul_qual": COUL_MAP[0],
                "date_dif": date_ech_str,
                "type_zone": TYPE_ZONE_DEFAULT,
                "code_zone": cz,
                "lib_zone": libz,
                "source": SOURCE_DEFAULT,
                # species codes
                "code_ambroisie": "0",
                "code_aulne": "0",
                "code_armoise": "0",
                "code_bouleau": "0",
                "code_graminees": "0",
                "code_olivier": "0",
                "x_wgs84": xw,
                "y_wgs84": yw,
                "x_reg": xr,
                "y_reg": yr,
                "epsg_reg": epsg
            }
            out_rows.append(row)
            continue

        # Measured day: determine an overall qualitative code (1..4)
        # seasonal factor: overall pollen load tends to be higher in spring-summer
        month = dt.month
        # basic seasonal multiplier ([~0.6 in winter .. ~1.6 in peak pollen season])
        seasonal_mult = 0.6 + 1.0 * math.exp(-((month - 6) ** 2) / (2 * 4.0))
        # zone-specific random jitter and occasional spikes
        spike = 1.0
        if random.random() < 0.003:  # rare big spike day
            spike = random.uniform(1.6, 2.5)
        jitter = random.uniform(0.85, 1.15)

        # compute a score where higher -> worse (4)
        score = ( (baseline - 1) / 3.0 ) * 0.4 + (seasonal_mult - 0.6) / 1.0 * 0.5
        score *= spike * jitter
        score += random.uniform(-0.12, 0.12)

        # map score to code_qual
        if score < 0.15:
            code_qual = 1
        elif score < 0.45:
            code_qual = 2
        elif score < 0.75:
            code_qual = 3
        else:
            code_qual = 4

        # Occasionally enforce low or high extremes for particular zones/days
        if random.random() < 0.002:
            code_qual = random.choice([1,4])

        lib_qual = LIB_QUAL_MAP[code_qual]
        coul = COUL_MAP.get(code_qual, "#000000")

        # species codes using seasonal activity and overall quality
        species_codes = {}
        for sp in SPECIES:
            act = season_activity(sp, dt)
            code = activity_to_code(act, code_qual)
            # add noise: some low-probability anomalies
            if random.random() < 0.01:
                code = max(1, min(4, code + random.choice([-1,1])))
            species_codes[sp] = code

        row = {
            "date_ech": date_ech_str,
            "code_qual": str(code_qual),
            "lib_qual": lib_qual,
            "coul_qual": coul,
            "date_dif": date_ech_str,
            "type_zone": TYPE_ZONE_DEFAULT,
            "code_zone": cz,
            "lib_zone": libz,
            "source": SOURCE_DEFAULT,
            "code_ambroisie": str(species_codes["ambroisie"]),
            "code_aulne": str(species_codes["aulne"]),
            "code_armoise": str(species_codes["armoise"]),
            "code_bouleau": str(species_codes["bouleau"]),
            "code_graminees": str(species_codes["graminees"]),
            "code_olivier": str(species_codes["olivier"]),
            "x_wgs84": xw,
            "y_wgs84": yw,
            "x_reg": xr,
            "y_reg": yr,
            "epsg_reg": epsg
        }
        out_rows.append(row)

# ----------------------
# Save CSV
# ----------------------
df_out = pd.DataFrame(out_rows, columns=[
    "date_ech","code_qual","lib_qual","coul_qual","date_dif","type_zone",
    "code_zone","lib_zone","source",
    "code_ambroisie","code_aulne","code_armoise","code_bouleau","code_graminees","code_olivier",
    "x_wgs84","y_wgs84","x_reg","y_reg","epsg_reg"
])

# Make sure numeric columns are represented the way you prefer; keep textual codes as strings
df_out.to_csv(OUTPUT_CSV, index=False, float_format="%.12g")
print(f"Generated file written to {OUTPUT_CSV}. Rows: {len(df_out)}")


Reading zones from existing CSV indice_pollen_bretagne_epci.csv ...
Using 62 zones for generation.
Generating daily measurements (this may take a little while)...


100%|██████████| 62/62 [00:01<00:00, 38.23zone/s]


Generated file written to indice_pollen_bretagne_epci_generated.csv. Rows: 135904
